# Agent Tools and Configurations

Tools give agents ability to take actions: sequential/parallel calls, error handling, and state persistence.

**What you'll learn:**
- Tools extend agent capabilities with actions
- @wrap_tool_call handles errors gracefully
- Agent selects tools based on docstrings
- Sequential vs parallel execution determined by model
- Clear tool descriptions guide selection

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [ ]:
# Custom Tool Name and Description
base_tools.web_search.name

base_tools.web_search.description

## Basic Agent with Tools

In [ ]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather]
)

response = agent.invoke({
    'messages': [HumanMessage("What's the weather in Tokyo?")]
})

response['messages'][-1].text

In [ ]:
# Agent without tools - single LLM node
agent = create_agent(model=model, tools=[])

response = agent.invoke({
    'messages': [HumanMessage("Hello")]
})

response['messages'][-1].text

## Sequential Tool Calls

In [ ]:
# Agent calls tools in sequence
response = agent.invoke({
    'messages': [HumanMessage(
        "Search for Apple news, then tell me weather in Cupertino"
    )]
})

response['messages'][-1].text

## Parallel Tool Calls

In [ ]:
# Agent may call tools in parallel (independent requests)
response = agent.invoke({
    'messages': [HumanMessage(
        "What's the weather in Paris and London?"
    )]
})

response['messages'][-1].text

## Tool Error Handling

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from langchain.tools import tool
# Tool WITHOUT internal error handling
@tool
def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b  # This will crash on division by zero

# Middleware catches it and returns graceful error to model
@wrap_tool_call
def handle_tool_errors(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}. Try different input.",
            tool_call_id=request.tool_call["id"]
        )


agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather, divide],
    middleware=[handle_tool_errors]
)

response = agent.invoke({
    'messages': [HumanMessage("what is 1/0?")]
})

response['messages'][-1].text

## Accessing Context - State Access

In [ ]:
from langchain.tools import ToolRuntime
from langchain_core.messages import AIMessage

# Tool that retrieves the count of messages in conversation
@tool
def get_message_count(runtime: ToolRuntime) -> str:
    """Get the total number of messages exchanged in the conversation."""
    messages = runtime.state["messages"]
    return f"Total messages in conversation: {len(messages)}"

@tool
def get_token_count(runtime: ToolRuntime) -> str:
    """Get the actual token usage from AI responses."""
    messages = runtime.state["messages"]
    
    # Track token usage from AI messages
    total_input_tokens = 0
    total_output_tokens = 0
    total_tokens = 0
    response_count = 0
    
    for msg in messages:
        if isinstance(msg, AIMessage):
            # Extract usage_metadata if available
            if hasattr(msg, 'usage_metadata') and msg.usage_metadata:
                total_input_tokens += msg.usage_metadata.get('input_tokens', 0)
                total_output_tokens += msg.usage_metadata.get('output_tokens', 0)
                total_tokens += msg.usage_metadata.get('total_tokens', 0)
                response_count += 1
    
    return f"AI responses: {response_count}\nInput tokens: {total_input_tokens}\nOutput tokens: {total_output_tokens}\nTotal tokens: {total_tokens}"

agent = create_agent(
    model=model,
    tools=[get_message_count, get_token_count]
)

response = agent.invoke({
    'messages': [
        HumanMessage("Let's talk about Python programming")
    ]
})

response['messages'][-1].text